# Preparación del dataset

En este notebook crearemos el dataset que será nuestro corpus para entrenar, ajustar y evaluar los modelos de detección automática de idiomas que se desarrollarán posteriormente. Nos ajustaremos a las directrices indicadas:
 
> - Seleccionar al menos 7 idiomas diferentes del corpus  
> - Extraer entre 1.000 y 10.000 frases por idioma, garantizando un balanceo entre las clases  
> - Controlar la longitud de las frases:  
>     - Mínimo: 2-3 palabras  
>     - Máximo: 15-20 palabras  
>     - Media objetivo: 6-7 palabras  
> - Dividir adecuadamente los datos en conjuntos de entrenamiento (70%), validación (15%) y prueba (15%)  
> - Documentar todo el proceso de preparación de datos  


In [1]:
# Importamos las librererías necesarias

import os
import requests
import tarfile
import re
import random 
import pandas as pd

Definiremos una función que nos permitirá descargar y procesar las urls proporcionadas para crear nuestro dataset

In [6]:
# Esta primera función será la que empleemos para tomar frases aleatorias de los archivos descargados, de acuerdo a nuestras restricciones de longitud

def read_file(path, n_sentences, random_seed=None, min_len = 3, max_len=1):
    """
    Lee un archivo de texto y devuelve un DataFrame con una muestra aleatoria de frases 
    que cumplen con una longitud específica de palabras.

    Args:
    - path (str): Ruta del archivo de texto a leer.
    - n_sentences (int): Número de frases que se desean extraer.
    - random_seed (int): Semilla aleatoria para hacerlo reproducible
    - min_len (int, opcional): Número mínimo de palabras por frase (por defecto, 3).
    - max_len (int, opcional): Número máximo de palabras por frase (por defecto, 15).

    Values:
    - pd.DataFrame: Contiene las frases seleccionadas, el idioma y el número de palabras.
    """

    with open(path, "r", encoding = "utf-8") as f:
        lines = f.readlines()
    
    # Filtramos las que cumplen nuestra condición de tamaño
    filtered_lines = [
    (s, len(words)) for s in lines
    if (words := re.findall(r'\b[^\W\d_]+\b', s, flags=re.UNICODE)) and min_len <= len(words) <= max_len
    ]   
    # Esta expresión regular retiene las palabras (secuencias de caracteres UNICODE
    # que no se que no sean dígitos, signos de putuación, espacios o "_", i.e. que sean 
    # letras, limitados por "word boundaries" como espacios o signos de puntuación)
    # Esta es solo una primera aproximación para hacer un primer filtrado

    # Tomamos una muestra aleatoria de tamaño n_rows
    if random_seed:
        random.seed(random_seed) 
    sample = random.sample(filtered_lines,n_sentences)

    # Separamos las frases y la longitud
    sentences, n_words = zip(*sample)

    # Creamos el dataframe
    _,ext = os.path.splitext(path)
    df = pd.DataFrame({"text": sentences,
                       "language": [ext.replace(".","") for _ in range(n_sentences)],
                       "n_words": n_words})
    return df

# Esta será la función que efectivamente cree el dataset

def get_dataset(urls, dir=None, download_folder=None, download_files=True, del_extra = True, n_sentences=5000, min_len=2, max_len=15, random_seed=None):
    """
    Descarga, descomprime y procesa archivos de texto multilingües desde una lista de URLs para construir un dataset.

    Args:
    - urls (list): Lista de URLs de los archivos a descargar.
    - dir (str): Carpeta base donde se guardará el dataset (por defecto: "multilingual_europarl_dataset").
    - download_folder (str): Subcarpeta donde se almacenan los archivos descargados (por defecto: "europarl_datasets").
    - download_files (bool): Si True, descarga los archivos desde las URLs proporcionadas.
    - random_seed (int): Semilla aleatoria para hacerlo reproducible
    - del_extra (bool): Si True, elimina los archivos comprimidos después de descomprimirlos y los archivos en inglés.
    - n_sentences (int): Número máximo de oraciones a leer por archivo.
    - min_len (int): Longitud mínima de oración permitida.
    - max_len (int): Longitud máxima de oración permitida.

    Values:
    - DataFrame de pandas con dos columnas: 'text' (oraciones) y 'language' (idioma).
    """

    if not dir:
        dir = "multilingual_europarl_dataset"
        if not os.path.exists(dir):
            os.makedirs(dir, exist_ok=True)
    
    if not download_folder:
        download_folder = "europarl_datasets"
    
    download_path = os.path.join(dir,download_folder)

    if download_files:
       if not os.path.exists(download_path):
           os.makedirs(download_path, exist_ok=True)

       # Procesar cada URL
       for url in urls:
           file_name = os.path.basename(url)
           file_path = os.path.join(download_path, file_name)

           # Descargar el archivo
           print(f"Descargando: {url}")
           resp = requests.get(url)
           with open(file_path, 'wb') as f:
               f.write(resp.content)

           # Descomprimir si es .tgz o .tar.gz
           if tarfile.is_tarfile(file_path):
                print(f"Descomprimiendo: {file_name}")
                with tarfile.open(file_path, 'r:gz') as tar:
                       tar.extractall(path=download_path, 
                                      filter=lambda tarinfo, path: tarinfo if not tarinfo.name.endswith(".en") else None)

           # Borramos archivos innecesarios
           if del_extra:
                os.remove(file_path)


    # Creamos el dataset a partir de los archivos descargados
    print("Creando el dataframe...")

    df_all = pd.DataFrame(columns=["text", "language", "n_words"])
    for file in os.listdir(download_path):
        extension = os.path.splitext(file)[1]
        if extension not in [".tgz",".en"]:
             print("  Procesando: ", file)
             df_lan = read_file(os.path.join(download_path, file), 
                                n_sentences=n_sentences,
                                min_len=min_len,
                                max_len=max_len,
                                random_seed=random_seed)
             
        df_all = pd.concat([df_all, df_lan], ignore_index=True)

    return df_all

Empleamos la función para crear nuestro dataset.
- Idiomas: 
    - Alemán
    - Español
    - Italiano
    - Francés
    - Esloveno
    - Portugués
    - Sueco
    - Checo
    - Danés
    - Polaco
    - Griego
    - Finés
- Número de frases por idioma: 5000
- Número mínimo de palabras por frase: 2
- Número máximo de palabras por frase: 15

Observación: Estamos empleando FRASES COMPLETAS, podría ser útil emplear también frases incompletas ya que puede ser una caso común de uso del detector de idioma (por ejemplo, si el usuario está escribiendo una frase para traducir puede ser útil poder detectar el idioma antes de que concluya). Estudiaremos si es necesario incluir en nuestro dataset frases incompletas.

In [11]:
# urls = ["https://www.statmt.org/europarl/v7/de-en.tgz", # Alemán
#         "https://www.statmt.org/europarl/v7/es-en.tgz", # Español
#         "https://www.statmt.org/europarl/v7/it-en.tgz", # Italiano
#         "https://www.statmt.org/europarl/v7/fr-en.tgz", # Francés
#         "https://www.statmt.org/europarl/v7/sl-en.tgz", # Esloveno
#         "https://www.statmt.org/europarl/v7/pt-en.tgz", # Portugués
#         "https://www.statmt.org/europarl/v7/sv-en.tgz"] # Sueco

urls = ["https://www.statmt.org/europarl/v7/de-en.tgz", # Alemán
        "https://www.statmt.org/europarl/v7/es-en.tgz", # Español
        "https://www.statmt.org/europarl/v7/it-en.tgz", # Italiano
        "https://www.statmt.org/europarl/v7/fr-en.tgz", # Francés
        "https://www.statmt.org/europarl/v7/sl-en.tgz", # Esloveno
        "https://www.statmt.org/europarl/v7/pt-en.tgz", # Portugués
        "https://www.statmt.org/europarl/v7/sv-en.tgz", # Sueco
        "https://www.statmt.org/europarl/v7/cs-en.tgz", # Checo
        "https://www.statmt.org/europarl/v7/da-en.tgz", # Danés
        "https://www.statmt.org/europarl/v7/pl-en.tgz", # Polaco
        "https://www.statmt.org/europarl/v7/el-en.tgz", # Griego
        "https://www.statmt.org/europarl/v7/fi-en.tgz"] # Finés 

data = get_dataset(urls, dir = os.path.join("data","raw"), 
                   download_files=True,
                   n_sentences=5000)
data.to_csv(os.path.join("data","raw","multilingual_europarl_dataset.csv"),
            index=False)

Descargando: https://www.statmt.org/europarl/v7/de-en.tgz
Descomprimiendo: de-en.tgz
Descargando: https://www.statmt.org/europarl/v7/es-en.tgz
Descomprimiendo: es-en.tgz
Descargando: https://www.statmt.org/europarl/v7/it-en.tgz
Descomprimiendo: it-en.tgz
Descargando: https://www.statmt.org/europarl/v7/fr-en.tgz
Descomprimiendo: fr-en.tgz
Descargando: https://www.statmt.org/europarl/v7/sl-en.tgz
Descomprimiendo: sl-en.tgz
Descargando: https://www.statmt.org/europarl/v7/pt-en.tgz
Descomprimiendo: pt-en.tgz
Descargando: https://www.statmt.org/europarl/v7/sv-en.tgz
Descomprimiendo: sv-en.tgz
Descargando: https://www.statmt.org/europarl/v7/cs-en.tgz
Descomprimiendo: cs-en.tgz
Descargando: https://www.statmt.org/europarl/v7/da-en.tgz
Descomprimiendo: da-en.tgz
Descargando: https://www.statmt.org/europarl/v7/pl-en.tgz
Descomprimiendo: pl-en.tgz
Descargando: https://www.statmt.org/europarl/v7/el-en.tgz
Descomprimiendo: el-en.tgz
Descargando: https://www.statmt.org/europarl/v7/fi-en.tgz
Descomp

## 

## Partición en entrenamiento/validación/test
Vamos a dividir el conjunto de datos en entrenamiento, validación y test. Para ello, definamos la siguiente función `split_dataset`

In [12]:
from sklearn.model_selection import train_test_split

def split_dataset(df, stratify_col=None, train_size=0.7, val_size=0.15, test_size=0.15, random_state=None):
    """
    Divide un DataFrame en conjuntos de entrenamiento, validación y test. Si val_size es nulo, 
    en entrenamiento y test.

    Args:
    - df (pd.DataFrame): El DataFrame a dividir.
    - stratify_col (str, opcional): Columna para estratificación.
    - train_size (float): Proporción del conjunto de entrenamiento (por defecto, 0.7).
    - val_size (float): Proporción del conjunto de validación (por defecto, 0.15).
    - test_size (float): Proporción del conjunto de prueba (por defecto, 0.15).
    - random_state (int): Semilla para reproducibilidad.

    Values:
    - train_df, val_df, test_df (pd.DataFrame): Conjuntos de entrenamiento, validación y prueba (si val_size > 0).
    - train_df, test_df (pd.Dataframe): Conjuntos de entrenamiento y test (si val_size = 0)
    """
    assert abs(train_size + val_size + test_size - 1.0) < 1e-5, "Las proporciones deben sumar 1."

    stratify_data = df[stratify_col] if stratify_col else None

    if val_size > 0:
        # Primera división: entrenamiento y temporal (val + test)
        train_df, temp_df = train_test_split(
            df, test_size=(1 - train_size), random_state=random_state, stratify=stratify_data
        )

        # Actualizamos la columna de estratificación
        stratify_temp = temp_df[stratify_col] if stratify_col else None

        # División de temporal en validación y prueba
        relative_test_size = test_size / (val_size + test_size)  # proporcional dentro del 30%
        val_df, test_df = train_test_split(
            temp_df, test_size=relative_test_size, random_state=random_state, stratify=stratify_temp
        )
        return train_df, val_df, test_df
    
    else:
        # Si val_size=0: train / test
        train_df, test_df = train_test_split(
            df, test_size=test_size, random_state=random_state, stratify=stratify_data
        )

        return train_df, test_df
    

Empleamos la función para crear nuestra partición de los datos y los guardamos para emplear siempre la misma

In [13]:
df_train, df_val, df_test = split_dataset(data, random_state=123)
df_train.to_csv(os.path.join("data", "raw","train.csv"), index=False)
df_val.to_csv(os.path.join("data", "raw","val.csv"), index=False)
df_test.to_csv(os.path.join("data", "raw","test.csv"), index=False)